In [1]:
import pandas as pd
import h2o,os
import numpy as np
import matplotlib.pyplot as plt
h2o.init()
import warnings
warnings.filterwarnings('ignore')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,29 mins 30 secs
H2O cluster timezone:,Africa/Nairobi
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_root_91eopm
H2O cluster total nodes:,1
H2O cluster free memory:,2.902 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [2]:
df =pd.read_csv('LoanStats3a.csv')
df.shape
#os.listdir()
df.columns
useful =['loan_amnt','funded_amnt','installment','int_rate',
         'term','emp_length','home_ownership','annual_inc','purpose','loan_status']
df =df[useful]
df.head()

,loan_amnt,funded_amnt,installment,int_rate,term,emp_length,home_ownership,annual_inc,purpose,loan_status
0,5000.0,5000.0,162.87,10.65%,36 months,10+ years,RENT,24000.0,credit_card,Fully Paid
1,2500.0,2500.0,59.83,15.27%,60 months,< 1 year,RENT,30000.0,car,Charged Off
2,2400.0,2400.0,84.33,15.96%,36 months,10+ years,RENT,12252.0,small_business,Fully Paid
3,10000.0,10000.0,339.31,13.49%,36 months,10+ years,RENT,49200.0,other,Fully Paid
4,3000.0,3000.0,67.79,12.69%,60 months,1 year,RENT,80000.0,other,Fully Paid


In [3]:
loans =df.copy()
loans = loans[(loans['loan_status']=='Fully Paid') | (loans['loan_status']=='Charged Off')]
loans['loan_status_bin'] = loans['loan_status'].map({'Charged Off': 1, 'Fully Paid': 0}) # 0 good loans 1 bad loans
loans["int_rate"]=[float(i.replace("%","")) for i in loans.int_rate]
loans['term'] =[int(i[:3]) for i in loans.term]
loans.head()
loans.isnull().sum()
loans = loans.fillna(0)
loans.isnull().sum()

loan_amnt          0
funded_amnt        0
installment        0
int_rate           0
term               0
emp_length         0
home_ownership     0
annual_inc         0
purpose            0
loan_status        0
loan_status_bin    0
dtype: int64

In [4]:
loans['emp_length'] = [str(i).replace('years',"") for i in loans['emp_length'] ]
#loans['emp_length'] = loans['emp_length'].fillna(0)
loans['emp_length'] = [str(i).replace('year',"") for i in loans['emp_length'] ]
loans['emp_length'] = [str(i).replace('<',"") for i in loans['emp_length'] ]
loans['emp_length'] = [str(i).replace('+',"") for i in loans['emp_length'] ]

loans['emp_length'] = [int(i) for i in loans['emp_length'] ]

loans.emp_length.value_counts()


10    8899
1     7837
2     4394
3     4098
4     3444
5     3286
6     2231
7     1775
8     1485
9     1259
0     1078
Name: emp_length, dtype: int64

In [5]:
#balancing
loans.loan_status_bin.value_counts()

# Balancing the data ti improve overall performance
filter1 = loans.loan_status_bin == 0
filter2 = loans.loan_status_bin == 1
good_loans = loans[filter1]
bad_loans = loans[filter2]
slices = good_loans.iloc[:5670,:]
new = pd.concat((bad_loans,slices),0)
print(good_loans.shape,bad_loans.shape,new.shape)
new.loan_status_bin.value_counts()
new.head()
new['loan_status_bin'] =["Good" if i==0 else 'bad' for i in new['loan_status_bin']]
new = new.sample(frac=1)#randomize the data
new.head()

(34116, 11) (5670, 11) (11340, 11)


,loan_amnt,funded_amnt,installment,int_rate,term,emp_length,home_ownership,annual_inc,purpose,loan_status,loan_status_bin
2626,3650.0,3650.0,91.86,17.58,60,4,RENT,29000.0,debt_consolidation,Fully Paid,Good
5296,10800.0,10800.0,381.25,16.29,36,9,RENT,75000.0,other,Charged Off,bad
11840,18225.0,15150.0,384.63,17.99,60,6,MORTGAGE,42000.0,debt_consolidation,Charged Off,bad
38595,12000.0,12000.0,407.17,13.49,36,1,RENT,55000.0,debt_consolidation,Charged Off,bad
19872,5000.0,5000.0,116.79,14.17,60,3,RENT,72000.0,credit_card,Charged Off,bad


In [6]:
#create the model
x=['loan_amnt', 'funded_amnt', 'installment', 'int_rate', 'term',
       'emp_length', 'home_ownership', 'annual_inc', 'purpose', 
       ]
N =h2o.H2OFrame(new)
train,test,valid =N.split_frame([0.8,0.1])
from h2o.estimators import H2OGradientBoostingEstimator as GB
from h2o.estimators import H2ORandomForestEstimator as RF
from h2o.estimators import H2ODeepLearningEstimator as DL

model1 = RF()
model =GB()
model2 = DL()
model2.train(x =x,y ='loan_status_bin',training_frame =train,validation_frame =test)
model1.train(x =x,y ='loan_status_bin',training_frame =train,validation_frame =test)
model.train(x =x,y ='loan_status_bin',training_frame =train,validation_frame =test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [7]:
#model1.plot()
#model1.varimp_plot()
pred =model1.predict(test)
print(model.confusion_matrix())
print(model1.confusion_matrix())
print(model2.confusion_matrix())
#model1.roc

drf prediction progress: |████████████████████████████████████████████████| 100%
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4872914410321741: 


,Good,bad,Error,Rate
Good,3771.0,768.0,0.1692,(768.0/4539.0)
bad,660.0,3874.0,0.1456,(660.0/4534.0)
Total,4431.0,4642.0,0.1574,(1428.0/9073.0)



Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.375073910525648: 


,Good,bad,Error,Rate
Good,2468.0,2071.0,0.4563,(2071.0/4539.0)
bad,623.0,3911.0,0.1374,(623.0/4534.0)
Total,3091.0,5982.0,0.2969,(2694.0/9073.0)



Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35345365313511906: 


,Good,bad,Error,Rate
Good,1822.0,2717.0,0.5986,(2717.0/4539.0)
bad,575.0,3959.0,0.1268,(575.0/4534.0)
Total,2397.0,6676.0,0.3628,(3292.0/9073.0)


In [32]:
u =h2o.save_model(model,path="/home/lostvane/Projects/school-project/CreditScore/app/ml_model/model")
u

'/home/lostvane/Projects/school-project/CreditScore/app/ml_model/model/GBM_model_python_1565074809758_307'

In [9]:
silly = h2o.load_model(u)

In [10]:
loan_amnt =1000
funded_amnt =1000 
installment =2 
int_rate =10 
term =36
emp_length =10
home_ownership ="OWN" 
annual_inc =30000
purpose = "car" 
kamau =[loan_amnt,funded_amnt,installment,int_rate,term,emp_length,home_ownership,annual_inc,purpose]
B =pd.DataFrame([kamau],columns =['loan_amnt', 'funded_amnt', 'installment', 'int_rate', 'term','emp_length', 'home_ownership', 'annual_inc', 'purpose'])
B = h2o.H2OFrame(B)
B
silly.predict(B)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,Good,bad
bad,0.0971677,0.902832


In [11]:
new.head()
#new.columns

,loan_amnt,funded_amnt,installment,int_rate,term,emp_length,home_ownership,annual_inc,purpose,loan_status,loan_status_bin
2626,3650.0,3650.0,91.86,17.58,60,4,RENT,29000.0,debt_consolidation,Fully Paid,Good
5296,10800.0,10800.0,381.25,16.29,36,9,RENT,75000.0,other,Charged Off,bad
11840,18225.0,15150.0,384.63,17.99,60,6,MORTGAGE,42000.0,debt_consolidation,Charged Off,bad
38595,12000.0,12000.0,407.17,13.49,36,1,RENT,55000.0,debt_consolidation,Charged Off,bad
19872,5000.0,5000.0,116.79,14.17,60,3,RENT,72000.0,credit_card,Charged Off,bad


In [14]:
u

'/good/GBM_model_python_1565074809758_307'

In [15]:
loans.head()
list(loans.iloc[1,:])

[2500.0, 2500.0, 59.83, 15.27, 60, 1, 'RENT', 30000.0, 'car', 'Charged Off', 1]

In [16]:

a =silly.predict(test[1:5,:])


gbm prediction progress: |████████████████████████████████████████████████| 100%


In [17]:
proba = a[0,1]
a[0,:]

predict,Good,bad
bad,0.191424,0.808576


In [18]:
proba

0.19142436715354338

In [19]:
v =silly.varimp()
df =pd.DataFrame(v,columns =['c1','c2','c3','c4'])
x =pd.melt(df,id_vars = 'c1')
import plotnine
from plotnine import *
(ggplot(x,aes(x="c1",y='value'))+geom_bar(stat ="identity",fill ="blue")+coord_flip()+labs(title="Variable importance"))

ModuleNotFoundError: No module named 'plotnine'

In [20]:
BB =['loan_amnt', 'installment', 'int_rate', 'term','emp_length', 'home_ownership', 'annual_inc', 'purpose',]
F=good_loans[BB]
F=h2o.H2OFrame(F)
x =['installment', 'int_rate', 'term','emp_length', 'home_ownership', 'annual_inc', 'purpose',]
train,test,valid =F.split_frame([0.9,0.05])
reg =GB()
reg.train(training_frame=train,validation_frame=test,x =x,y='loan_amnt')



Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [21]:
G =test[:2,:]
G=G[x]
reg.predict(G)


gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
7305.97
9454.87


In [22]:
test[:2,:]

loan_amnt,installment,int_rate,term,emp_length,home_ownership,annual_inc,purpose
7100,252.33,16.77,36,1,MORTGAGE,33000,debt_consolidation
9200,307.43,12.42,36,2,RENT,60000,debt_consolidation


In [23]:
K = h2o.H2OFrame(bad_loans.iloc[40:50,:])
K =K[x]
reg.predict(K)

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict
11044.8
15143.6
9637
7365.06
25079.5
7331.86
10003.1
17420.1
4069.95
10087.3


In [24]:
bad_loans.iloc[40:50,:]

,loan_amnt,funded_amnt,installment,int_rate,term,emp_length,home_ownership,annual_inc,purpose,loan_status,loan_status_bin
204,10500.0,10500.0,370.66,16.29,36,3,RENT,30000.0,debt_consolidation,Charged Off,1
212,15075.0,15075.0,517.21,14.27,36,10,RENT,35000.0,debt_consolidation,Charged Off,1
220,9500.0,9500.0,318.68,12.69,36,1,MORTGAGE,52500.0,moving,Charged Off,1
224,7200.0,7200.0,225.29,7.90,36,3,RENT,68000.0,debt_consolidation,Charged Off,1
225,26000.0,26000.0,642.96,16.77,60,10,MORTGAGE,53000.0,home_improvement,Charged Off,1
226,7000.0,7000.0,154.69,11.71,60,6,MORTGAGE,69600.0,credit_card,Charged Off,1
231,10000.0,10000.0,335.45,12.69,36,2,RENT,70000.0,credit_card,Charged Off,1
239,15600.0,15600.0,352.48,12.69,60,10,RENT,50000.0,vacation,Charged Off,1
251,4000.0,4000.0,134.18,12.69,36,2,RENT,25000.0,debt_consolidation,Charged Off,1
255,10000.0,10000.0,334.16,12.42,36,10,RENT,39000.0,credit_card,Charged Off,1


In [25]:
banks = ['KCB BANK', 'EQUITY BANK','COOPERATIVE BANK','FAMILY BANK','STANBIC', 'GITHIRI','MSHWARI']
loans = ['Fuzu loan', 'Jiajiri loan', 'Chama loan', 'Chetu loans', 'Tusome loans','loan on items','mobile loan']
rates = [14,12,15,7,4,20,80]
import random
max_amount = [random.randint(1000, 10000) for i in range(len(banks))]

empty = pd.DataFrame()
empty['BANKS']=banks
empty['LOANS']=loans
empty['RATES'] = rates
empty['MAX_AMOUNT'] = max_amount

empty.columns
pd.DataFrame(columns =['BANKS', 'LOANS', 'RATES', 'MAX_AMOUNT'])

,BANKS,LOANS,RATES,MAX_AMOUNT


In [26]:
def trump(x,df):
    if x*100<21:
        return pd.DataFrame()
    elif x*100<30:
        fil=df.RATES>14
        fil1 =df.RATES<20
        return df[fil &fil1 ]
    elif x*100<40:
        fil=df.RATES>=10
        fil1 =df.RATES<14
        return df[fil &fil1 ]
    elif x*100<50:
        fil=df.RATES>5
        fil1 =df.RATES<10
        return df[fil &fil1 ]
    else:
        filters =df.RATES<5
        return df[filters]
B =trump(0.22,empty)
B.shape[0]
banks =[i for i in B.BANKS[:2]]
loans =[i for i in B.LOANS[:2]]
interest =[i for i in B.RATES[:2]]
maxi =[i for i in B.MAX_AMOUNT[:2]]
banks
loans
interest
maxi

[7142]

In [27]:
B.BANKS

2    COOPERATIVE BANK
Name: BANKS, dtype: object

In [28]:
empty.to_csv('loans.csv')